## Assessing the impact of remote work on workers' mental health

##### Created by: Finn Pham
##### Date: November 3rd, 2024

### INTRODUCTION AND METHODS:
Since the beginning of the COVID-19 pandemic, the concept of remote work where professionals were able to perform their work duties from the comfort of their home has become more prevalent. According to an article by ConnexOntario (2023) on the relations between remote work and mental health, while remote work allieviates the commute stress as well as offering more flexibility for workers, this concept may bring about the decline of mental health. In the analysis, the author inditified some common drawbacks of remote work such as high rates of social isolation and lack of mental health resources.

In this project, I will be utilizing a fictional dataset provided on Kaggle (link: https://www.kaggle.com/datasets/waqi786/remote-work-and-mental-health), which contains information on workers' roles, their satisfaction of remote work as well as surrounding conditions. With the dataset, I am hoping to come up with a comprehensive analysis workflow with the following steps:
* Database connection and uploading using relevant frameworks
* Exploring the general demographic of the workers (based on their roles, age, industry, geographical locations and gender)
* Assessing workers' satisfactions using relevant indicators such as work-life balance rate, common mental health illness.
* Visualize results in Tableau.

As part of the project, I am planning to also hone skills in writing SQL queries and (perhaps learn how to write optimized scripts)

Link to Tableau dashboard: [COMING SOON]

### DATABASE CONNECTION AND LOADING:
First, we will be forming a conection with the MySQL Workbench database using `pymysql`

In [1]:
#Loading required extension
%load_ext sql

In [2]:
import pandas as pd
import pymysql

In [3]:
##Connect with database
db_name = "mental_health_db"
db_host = "localhost"
db_username = "root"
db_password = "_________" ##Hidden due to privacy issue

try:
    conn = pymysql.connect(host = db_host,
                          port = int(3306),
                          user = "root",
                          password = db_password,
                          db = db_name)
except e:
    print(e)
if conn:
    print("Connection Successful")
else:
    print("error")

Connection Successful


The connection was notified to be successful. We will be proceeding with data loading

In [4]:
mental_health_df = pd.read_sql_query("""
                                SELECT *
                                FROM impact_of_remote_work_on_mental_health
                               """, conn)
mental_health_df.head(5)

C:\Users\Admin\AppData\Local\Temp\ipykernel_3620\144603819.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  mental_health_df = pd.read_sql_query("""


,Employee_ID,Age,Gender,Job_Role,Industry,Years_of_Experience,Work_Location,Hours_Worked_Per_Week,Number_of_Virtual_Meetings,Work_Life_Balance_Rating,Stress_Level,Mental_Health_Condition,Access_to_Mental_Health_Resources,Productivity_Change,Social_Isolation_Rating,Satisfaction_with_Remote_Work,Company_Support_for_Remote_Work,Physical_Activity,Sleep_Quality,Region
0,EMP0001,32,Non-binary,HR,Healthcare,13,Hybrid,47,7,2,Medium,Depression,No,Decrease,1,Unsatisfied,1,Weekly,Good,Europe
1,EMP0002,40,Female,Data Scientist,IT,3,Remote,52,4,1,Medium,Anxiety,No,Increase,3,Satisfied,2,Weekly,Good,Asia
2,EMP0003,59,Non-binary,Software Engineer,Education,22,Hybrid,46,11,5,Medium,Anxiety,No,No Change,4,Unsatisfied,5,None,Poor,North America
3,EMP0004,27,Male,Software Engineer,Finance,20,Onsite,32,8,4,High,Depression,Yes,Increase,3,Unsatisfied,3,None,Poor,Europe
4,EMP0005,49,Male,Sales,Consulting,32,Onsite,35,12,2,High,None,Yes,Decrease,3,Unsatisfied,3,Weekly,Average,North America


### DEMOGRAPHIC ANALYSIS:
In this section, I will be diving deep into the demography of the workers as part of the data exploration process. Some indicators in use: industry,occupations, age and gender. 

In [6]:
                                #Demography: What jobs and industries are available within the dataset
industry_dt = pd.read_sql("""
                            SELECT Industry, COUNT(*) AS industry_count
                            FROM impact_of_remote_work_on_mental_health
                            GROUP BY Industry
                            ORDER BY COUNT(*) DESC
                          """, conn)
industry_dt

C:\Users\Admin\AppData\Local\Temp\ipykernel_6524\1804448412.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  industry_dt = pd.read_sql("""


,Industry,industry_count
0,Finance,747
1,IT,746
2,Healthcare,728
3,Retail,726
4,Education,690
5,Manufacturing,683
6,Consulting,680


According to the output generated above, I notice that while the worker's industries are somewhat equally distributed. However, the two most common industries were __Finance__ and __Information Technology__ while there was less entries for __Manufacturing__ and __Consulting__

In [10]:

# What are the different types of jobs that are available within the dataset:
occupation_exploration_dt = pd.read_sql("""
                                        SELECT Job_Role, COUNT(*) AS occupation_count
                                        FROM impact_of_remote_work_on_mental_health
                                        GROUP BY Job_Role
                                        ORDER BY COUNT(*) DESC
                                        """, conn)

occupation_exploration_dt

C:\Users\Admin\AppData\Local\Temp\ipykernel_6524\530858715.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  occupation_exploration_dt = pd.read_sql("""


,Job_Role,occupation_count
0,Project Manager,738
1,Sales,733
2,Designer,723
3,HR,716
4,Software Engineer,711
5,Data Scientist,696
6,Marketing,683


Of all the occupations listed in the dataset, most workers reported their roles as Project Manager and Designer.

In [11]:
##Age demographic:
## Divide up into different age groups
age_demography_df = pd.read_sql("""
                                WITH age_categories AS (
                                    SELECT Age, 
                                           CASE WHEN Age >= 12 AND Age <= 27 THEN 'Gen Z'
                                                WHEN Age >= 28 AND Age <= 43 THEN 'Millenials'
                                                WHEN Age >= 44 AND Age <= 59 THEN 'Gen X'
                                                ELSE 'Other' END AS Age_Categories
                                    FROM impact_of_remote_work_on_mental_health
                                ) SELECT Age_Categories, COUNT(*) AS age_gr_count
                                FROM age_categories
                                GROUP BY Age_Categories
                                ORDER BY COUNT(*) DESC
                                """, conn)
age_demography_df

C:\Users\Admin\AppData\Local\Temp\ipykernel_6524\1373376500.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  age_demography_df = pd.read_sql("""


,Age_Categories,age_gr_count
0,Gen X,2070
1,Millenials,2011
2,Gen Z,794
3,Other,125


In the age demographic analysis, I divided the age groups into three groups (Gen Z, Millenials and Gen X) where I searched open sources and determined the thresholds for each group. From the results, Gen X workers (ages 44 - 59) accounted for the majority of the workers in dataset, followed by Millenials (age 28-43) and Gen Z (age 12 - 27)

In [12]:
#Region demography: Where do most workers come from?
region_df = pd.read_sql("""
                        SELECT Region, COUNT(*) AS region_count
                        FROM impact_of_remote_work_on_mental_health
                        GROUP BY Region
                        ORDER BY COUNT(*) DESC
                        """, conn)
region_df

C:\Users\Admin\AppData\Local\Temp\ipykernel_6524\785881889.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  region_df = pd.read_sql("""


,Region,region_count
0,Oceania,867
1,Africa,860
2,Europe,840
3,Asia,829
4,South America,827
5,North America,777


Most workers in the dataset reside in the Oceania, Africa and Europe regions.

### Assessing workers' satisfaction:

In [10]:
###Stress-related metrics:
#Level 1 Assessment: All factors equivalent, do remote workers report higher work-life balance?
work_life_balance_compare = pd.read_sql("""
                                        SELECT Work_Location, AVG(Work_Life_Balance_Rating) AS avg_wlb_rate
                                        FROM impact_of_remote_work_on_mental_health
                                        GROUP BY Work_Location
                                        ORDER BY AVG(Work_Life_Balance_Rating) DESC
                                        """, conn)

work_life_balance_compare

C:\Users\Admin\AppData\Local\Temp\ipykernel_3620\3453056551.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  work_life_balance_compare = pd.read_sql("""


,Work_Location,avg_wlb_rate
0,Hybrid,3.0224
1,Remote,2.9831
2,Onsite,2.9469


In [11]:
#Level 1 Assessment: Among each group of workers, what is the percentage of workers reporting they are satisfied with remote work?
remote_satisfaction = pd.read_sql("""
                                  WITH count_tb AS (
                                  SELECT DISTINCT Work_Location,
                                         Satisfaction_with_Remote_Work, 
                                         COUNT(*) OVER (PARTITION BY Work_Location) AS row_count,
                                         COUNT(Satisfaction_with_Remote_Work) OVER (PARTITION BY Work_Location, Satisfaction_with_Remote_Work) AS satisfaction_count
                                  FROM impact_of_remote_work_on_mental_health
                                  ) SELECT *,  
                                           ROUND((satisfaction_count / row_count)*100, 2) AS rating_pct
                                    FROM count_tb
                                  """, conn)

remote_satisfaction

C:\Users\Admin\AppData\Local\Temp\ipykernel_3620\3240450276.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  remote_satisfaction = pd.read_sql("""


,Work_Location,Satisfaction_with_Remote_Work,row_count,satisfaction_count,rating_pct
0,Hybrid,Neutral,1649,539,32.69
1,Hybrid,Satisfied,1649,564,34.20
2,Hybrid,Unsatisfied,1649,546,33.11
3,Onsite,Neutral,1637,506,30.91
4,Onsite,Satisfied,1637,592,36.16
5,Onsite,Unsatisfied,1637,539,32.93
6,Remote,Neutral,1714,603,35.18
7,Remote,Satisfied,1714,519,30.28
8,Remote,Unsatisfied,1714,592,34.54


In [12]:
#Level 2 Assessment: For the unsatisfied remote workers, what were the most common mental health condition?
mental_illness = pd.read_sql("""
                             SELECT Mental_Health_Condition, COUNT(*) AS condition_count
                             FROM impact_of_remote_work_on_mental_health
                             WHERE Work_Location = 'Remote' AND Satisfaction_with_Remote_Work = 'Unsatisfied'
                             GROUP BY Mental_Health_Condition
                             """, conn)
mental_illness

C:\Users\Admin\AppData\Local\Temp\ipykernel_3620\2615952515.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  mental_illness = pd.read_sql("""


,Mental_Health_Condition,condition_count
0,None,157
1,Depression,140
2,Burnout,148
3,Anxiety,147


In [13]:
#Level 2 Assessment: For the unsatisfied worker, how does sleep quality look like?
sleep_quality = pd.read_sql("""
                            SELECT Sleep_Quality, COUNT(*) AS sleep_quality_count 
                            FROM impact_of_remote_work_on_mental_health
                            WHERE Work_Location = 'Remote' AND Satisfaction_with_Remote_Work = 'Unsatisfied'
                            GROUP BY Sleep_Quality
                            """, conn)
sleep_quality

C:\Users\Admin\AppData\Local\Temp\ipykernel_3620\2487107664.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sleep_quality = pd.read_sql("""


,Sleep_Quality,sleep_quality_count
0,Poor,186
1,Average,215
2,Good,191


In [32]:
#Within each occupation, which role reported highest rate of unsatisfaction with remote work?
job_occupation_wlb = pd.read_sql("""
                                WITH count_tb AS (
                                  SELECT DISTINCT Job_Role,
                                         Satisfaction_with_Remote_Work, 
                                         COUNT(*) OVER (PARTITION BY Job_Role) AS role_count,
                                         COUNT(Satisfaction_with_Remote_Work) OVER (PARTITION BY Job_Role, Satisfaction_with_Remote_Work) AS satisfaction_count
                                  FROM impact_of_remote_work_on_mental_health
                                  WHERE Work_Location = 'Remote'
                                  ) SELECT *,  
                                           ROUND((satisfaction_count / role_count)*100, 2) AS rating_pct
                                    FROM count_tb
                                    WHERE Satisfaction_with_Remote_Work = 'Unsatisfied'
                                    ORDER BY rating_pct DESC
                                """, conn)

job_occupation_wlb

C:\Users\Admin\AppData\Local\Temp\ipykernel_3620\3134958314.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  job_occupation_wlb = pd.read_sql("""


,Job_Role,Satisfaction_with_Remote_Work,role_count,satisfaction_count,rating_pct
0,HR,Unsatisfied,255,97,38.04
1,Designer,Unsatisfied,243,90,37.04
2,Marketing,Unsatisfied,228,84,36.84
3,Data Scientist,Unsatisfied,254,88,34.65
4,Sales,Unsatisfied,245,82,33.47
5,Software Engineer,Unsatisfied,247,78,31.58
6,Project Manager,Unsatisfied,242,73,30.17


In [30]:
access_resource = pd.read_sql("""
                                WITH count_tb AS (
                                  SELECT DISTINCT Access_to_Mental_Health_Resources,
                                         Satisfaction_with_Remote_Work, 
                                         COUNT(*) OVER (PARTITION BY Access_to_Mental_Health_Resources) AS freq,
                                         COUNT(Satisfaction_with_Remote_Work) OVER (PARTITION BY Access_to_Mental_Health_Resources, Satisfaction_with_Remote_Work) AS satisfaction_count
                                  FROM (SELECT * FROM impact_of_remote_work_on_mental_health
                                        WHERE Work_Location = 'Remote') r
                                  ) SELECT *,  
                                           ROUND((satisfaction_count / freq)*100, 2) AS rating_pct
                                    FROM count_tb
                                    WHERE Satisfaction_with_Remote_Work = 'Unsatisfied'
                                    ORDER BY rating_pct DESC
                                """, conn)

access_resource

C:\Users\Admin\AppData\Local\Temp\ipykernel_3620\4226892932.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  access_resource = pd.read_sql("""


,Access_to_Mental_Health_Resources,Satisfaction_with_Remote_Work,freq,satisfaction_count,rating_pct
0,No,Unsatisfied,896,317,35.38
1,Yes,Unsatisfied,818,275,33.62
